In [ ]:
import pandas as pd
from radcad import Model, Simulation, Experiment

model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=params)
simulation = Simulation(model=model, timesteps=100_000, runs=1)

result = simulation.run()
# Or, multiple simulations:
# experiment = Experiment([simulation_1, simulation_2, simulation_3])
# result = experiment.run()

df = pd.DataFrame(result)

In [4]:
# Initial states
initial_state = {
    'fee_rate': 0.01,  # Initial continuous fee rate
    'validators': {},  # Dictionary to hold validator data
    'gas_fees': 0,     # Initial gas fees
    'incentives': 0    # Initial incentives
}

# Parameters
params = {
    'initial_fee_rate': 0.01,
    'dynamic_adjustment_factor': 0.1,
    'max_fee_rate': 0.05,
    'min_fee_rate': 0.005,
    'validator_registration_delay': 2  # Delay in weeks
}
def p_update_fee_rate(params, substep, state_history, previous_state):
    """
    Mechanism to update the continuous fee rate dynamically based on network conditions.
    """
    fee_rate = previous_state['fee_rate']
    adjustment = params['dynamic_adjustment_factor'] * (previous_state['gas_fees'] / 100)
    new_fee_rate = fee_rate + adjustment
    new_fee_rate = max(min(new_fee_rate, params['max_fee_rate']), params['min_fee_rate'])
    return {'fee_rate': new_fee_rate}

def p_register_validator(params, substep, state_history, previous_state, policy_input):
    """
    Process to model validator registration and delays.
    """
    validators = previous_state['validators']
    new_validator = policy_input['new_validator']
    registration_delay = params['validator_registration_delay']
    validators[new_validator] = registration_delay
    return {'validators': validators}

def s_apply_continuous_fee(params, step, sL, s, _input):
    """
    Apply the continuous fee to validators and update their incentives.
    """
    fee_rate = s['fee_rate']
    validators = s['validators']
    incentives = s['incentives']

    # Example fee application logic
    for validator in validators:
        incentives -= fee_rate

    return ('incentives', incentives)

def p_register_validator(params, substep, state_history, previous_state, policy_input):
    """
    Process to model validator registration and delays.
    """
    validators = previous_state['validators']
    new_validator = policy_input['new_validator']
    registration_delay = params['validator_registration_delay']
    validators[new_validator] = registration_delay
    return {'validators': validators}


def p_adjust_fee_rate(params, substep, state_history, previous_state):
    """
    Adjust the fee rate dynamically based on network conditions.
    """
    fee_rate = previous_state['fee_rate']
    gas_fees = previous_state['gas_fees']
    adjustment_factor = params['dynamic_adjustment_factor']

    # Example adjustment logic
    if gas_fees > 50:
        new_fee_rate = fee_rate + adjustment_factor
    elif gas_fees < 20:
        new_fee_rate = fee_rate - adjustment_factor
    else:
        new_fee_rate = fee_rate

    new_fee_rate = max(min(new_fee_rate, params['max_fee_rate']), params['min_fee_rate'])

    return {'fee_rate': new_fee_rate}

def s_distribute_incentives(params, step, sL, s, _input):
    """
    Distribute incentives to validators.
    """
    incentives = s['incentives']
    validators = s['validators']

    # Example incentive distribution logic
    for validator in validators:
        incentives += 1  # Add incentives for each validator

    return ('incentives', incentives)

def p_validator_behavior(params, substep, state_history, previous_state):
    """
    Model how validators react to different workloads and fees.
    """
    validators = previous_state['validators']
    fee_rate = previous_state['fee_rate']

    # Example behavior logic
    for validator in validators:
        if fee_rate > 0.03:
            validators[validator] -= 1  # Reduce incentives for high fee rates

    return {'validators': validators}

from cadCAD.configuration import Configuration
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

# Define the partial state update blocks
partial_state_update_blocks = [
    {
        'policies': {
            'adjust_fee_rate': p_adjust_fee_rate,
            'register_validator': p_register_validator,
            'validator_behavior': p_validator_behavior,
        },
        'variables': {
            'fee_rate': s_apply_continuous_fee,
            'validators': s_distribute_incentives,
            'incentives': s_distribute_incentives,
        }
    }
]

# Create the configuration
sim_config = Configuration(user_id=1,model_id=23,subset_id=12,subset_window=10,initial_state=initial_state, partial_state_update_blocks=partial_state_update_blocks, params=params)

# Set up the execution mode and context
exec_mode = ExecutionMode()
context = ExecutionContext(exec_mode.local_mode)
executor = Executor(context, [sim_config])

# Execute the simulation
raw_result, tensor_field = executor.execute()



                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.5.3
Execution Mode: local_proc


KeyError: 'T'